# SK Chatbot with plugins

- Adding SK plugins to an SK Chatbot provides similar functionality as OpenAI function calling:
  - During the conversation the bot will determine if it needs to call external functions, for example, to get the current weather or to get product information from a database.

References:

- [eShopOnAzure Demo](https://github.com/Azure-Samples/eShopOnAzure)
- [ChatState.cs](https://github.com/Azure-Samples/eShopOnAzure/blob/main/src/WebApp/Components/Chatbot/ChatState.cs)

In [ ]:
#r "nuget: Microsoft.SemanticKernel,1.16.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI,1.16.0"
#r "nuget: dotenv.net"

using System;
using System.Text.Json;
using System.ComponentModel;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using dotenv.net;

// Load environment variables
DotEnv.Load();
var azureEndpoint = Environment.GetEnvironmentVariable("ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("API_KEY");
var model = Environment.GetEnvironmentVariable("GPT_MODEL");

// Create SK Kernel builder
var builder = Kernel.CreateBuilder();

// Add OpenAI completion service to the kernel
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

// Build the kernel
var kernel = builder.Build();

// Get the chat service from the kernel
var chatService = kernel.GetRequiredService<IChatCompletionService>();

### Create a class with several functions and add them to the kernel

In [ ]:

private sealed class AppPluginService
{
    [KernelFunction, Description("Gets the weather information")]
    public string GetWeather([Description("The city for which to get the weather")] string city)
    {
        return $"The temperature in {city} is 80F.";
    }
    [KernelFunction, Description("Gets the product information")]
    public string ProductInfo([Description("The product Id number")] int itemId)
    {
        return $"Getting the product information for item {itemId} from the database";
    }
}

// Add the plugins to the kernel
kernel.Plugins.AddFromObject(new AppPluginService());

// Auto call the plugins
OpenAIPromptExecutionSettings _aiSettings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };


In [ ]:
// Create a chat history
ChatHistory messages = new();
messages.AddSystemMessage("You are a helpful assistant. One of the things you can do is get a weather report for a city and get product information.");

In [ ]:
// Print the messages in the history
void PrintMessages(ChatHistory messages)
{
    foreach (var message in messages)
    {
        if (!string.IsNullOrEmpty(message.Content))
            Console.WriteLine($"{message.Role}:\n{message.Content}\n");
    }
}

In [ ]:
// Use history trimmer to manage the LLM's context window
ChatHistory MessageTrimmer(ChatHistory messages, bool keepSystem=true, int history=2)
{
    ChatHistory trimmedMessages = new();
    var existsSystemRole = keepSystem && messages[0].Role == AuthorRole.System;
    if (keepSystem && existsSystemRole)
    {
        trimmedMessages.Add(messages[0]);
    }
    if (messages.Count > history*2 + (existsSystemRole ? 1 : 0))
    {
        for (int i = messages.Count - history; i < messages.Count; i++)
        {
            trimmedMessages.Add(messages[i]);
        }
    } else
    {
        trimmedMessages = messages;
    }
    return trimmedMessages;
}

In [ ]:
// Process a user message
async Task ProcessMessage(string input) {
    messages.AddUserMessage(input);
    ChatMessageContent response = await chatService.GetChatMessageContentAsync(MessageTrimmer(messages), _aiSettings, kernel);
    if (!string.IsNullOrEmpty(response.Content))
    {
        messages.Add(response);
    }
}

In [ ]:
// No function calling
await ProcessMessage("What is the speed of light?");
PrintMessages(messages);

In [ ]:
// Call the weather function
messages.Clear();
await ProcessMessage("Get the weather for Miami");
PrintMessages(messages);

In [ ]:
// Call the product information function
messages.Clear();
await ProcessMessage("Get the product information for item 123");
PrintMessages(messages);